### 2019~2020 영화별 관객수, 상영스크린 수, 상영횟수

In [2]:
import chart_studio
chart_studio.tools.set_credentials_file(username='god_yes_jeans', api_key="zGQWSZLZr5QC2C8kAU52")

In [3]:
import requests
import json
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

In [4]:
url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=d0e3a97920ed001b0f0dc6f0b3cc7ef5&targetDt=20190101"
res = requests.get(url)
text = res.text

d = json.loads(text)

In [5]:
d

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20190101~20190101',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20180290',
    'movieNm': '아쿠아맨',
    'openDt': '2018-12-19',
    'salesAmt': '3482411050',
    'salesShare': '33.8',
    'salesInten': '1154385750',
    'salesChange': '49.6',
    'salesAcc': '33936703431',
    'audiCnt': '384106',
    'audiInten': '96412',
    'audiChange': '33.5',
    'audiAcc': '3875963',
    'scrnCnt': '1158',
    'showCnt': '4594'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20170658',
    'movieNm': 'PMC: 더 벙커',
    'openDt': '2018-12-26',
    'salesAmt': '1763943100',
    'salesShare': '17.1',
    'salesInten': '425397650',
    'salesChange': '31.8',
    'salesAcc': '11468190467',
    'audiCnt': '195859',
    'audiInten': '30733',
    'audiChange': '18.6',
    'audiAcc': '1374890',
    'scrnCnt': '893

In [6]:
info = json_normalize(d['boxOfficeResult']['dailyBoxOfficeList'])

In [7]:
info.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20180290,아쿠아맨,2018-12-19,3482411050,33.8,1154385750,49.6,33936703431,384106,96412,33.5,3875963,1158,4594
1,2,2,0,OLD,20170658,PMC: 더 벙커,2018-12-26,1763943100,17.1,425397650,31.8,11468190467,195859,30733,18.6,1374890,893,3530
2,3,3,1,OLD,20185485,보헤미안 랩소디,2018-10-31,1236738450,12.0,378199650,44.1,81247178795,135879,29431,27.6,9360461,732,1872
3,4,4,-1,OLD,20186281,범블비,2018-12-25,1143504700,11.1,264548150,30.1,10899588452,132618,21611,19.5,1299768,766,2519
4,5,5,0,NEW,20184187,언니,2019-01-01,612798450,5.9,612798450,100,623953450,66686,66686,100,67929,509,1679


In [8]:
df = info[['movieCd','audiCnt','scrnCnt','showCnt']]

In [9]:
df.insert(0,'date','20190101')

In [10]:
df.head()

,date,movieCd,audiCnt,scrnCnt,showCnt
0,20190101,20180290,384106,1158,4594
1,20190101,20170658,195859,893,3530
2,20190101,20185485,135879,732,1872
3,20190101,20186281,132618,766,2519
4,20190101,20184187,66686,509,1679


In [11]:
def intodf(df,date):
    url = f'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=d0e3a97920ed001b0f0dc6f0b3cc7ef5&targetDt={date}'
    res = requests.get(url)
    text = res.text
    d = json.loads(text)
    info = json_normalize(d['boxOfficeResult']['dailyBoxOfficeList'])
    new = info[['movieCd','audiCnt','scrnCnt','showCnt']]
    new.insert(0,'date',date)
    df = pd.concat([df,new])
    return df

In [12]:
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

In [13]:
date2019=date_range("20190102", "20191231")
for d in date2019:
    df = intodf(df,d)

In [14]:
date2020=date_range("20200101", "20201231")
for d in date2020:
    df = intodf(df,d)

In [15]:
df

,date,movieCd,audiCnt,scrnCnt,showCnt
0,20190101,20180290,384106,1158,4594
1,20190101,20170658,195859,893,3530
2,20190101,20185485,135879,732,1872
3,20190101,20186281,132618,766,2519
4,20190101,20184187,66686,509,1679
...,...,...,...,...,...
5,20201231,20202647,2548,322,431
6,20201231,20206547,1872,51,58
7,20201231,20167904,1612,35,58
8,20201231,20202128,1327,158,169


In [17]:
df = df.reset_index()

In [18]:
df

,index,date,movieCd,audiCnt,scrnCnt,showCnt
0,0,20190101,20180290,384106,1158,4594
1,1,20190101,20170658,195859,893,3530
2,2,20190101,20185485,135879,732,1872
3,3,20190101,20186281,132618,766,2519
4,4,20190101,20184187,66686,509,1679
...,...,...,...,...,...,...
7305,5,20201231,20202647,2548,322,431
7306,6,20201231,20206547,1872,51,58
7307,7,20201231,20167904,1612,35,58
7308,8,20201231,20202128,1327,158,169


In [19]:
df.to_csv('movie.csv')